<a href="https://colab.research.google.com/github/dmbk/Anomaly-Detection-System/blob/master/FINAL_MODEL_AAE_Avenue.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
!pip install imageio
!pip install progress
#!pip install tensorflow_datasets
!pip install keras-layer-normalization
from google.colab import drive
#!pip install alive-progress
drive.mount('/content/drive', force_remount=True)


Mounted at /content/drive


In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals
try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass

import skimage
import glob
import imageio
import matplotlib.pyplot as plt
import numpy as np
import os
import PIL
from PIL import Image
from os.path import join
from os import listdir
from os.path import isfile, join, isdir

#import keras
import argparse
from os.path import dirname
import cv2
from google.colab.patches import cv2_imshow
	
import statistics
import shutil

import matplotlib.pyplot as plt

import time
from progress.bar import IncrementalBar

import numpy as np
from keras.models import Model, Sequential
from keras.layers import Input, Dense, Reshape, Flatten
from keras.layers import Conv2DTranspose, ConvLSTM2D, BatchNormalization, TimeDistributed, Conv2D, Dropout, Activation, InputLayer, MaxPool3D
from keras.optimizers import Adam
from keras_layer_normalization import LayerNormalization
from keras.models import load_model

In [0]:
class Config:
    def __init__(self, data_dir_, cwdir_name_, data_set):
        self.data_set_name = data_set
        self.data_dir = data_dir_
        self.data_set_dir = join(self.data_dir, data_set)
        self.cwdir_name = cwdir_name_
        self.cwdir = join(self.data_dir,self.cwdir_name)
        self.run_data = join(self.cwdir, "training_dir")
        self.image_dir = join(self.run_data,self.data_set_name,"testing_videos/")
        if not os.path.exists(self.cwdir):
            os.mkdir(self.cwdir)
            os.mkdir(self.run_data)
    
        if not os.path.exists(self.run_data):
            #shutil.rmtree(self.run_data)
            os.mkdir(self.run_data)
            os.makedirs(self.image_dir, exist_ok=True)

        self.DATASET_PATH = join(self.data_set_dir,"training_videos/")
        self.TEST_DIR = join(self.data_set_dir,"testing_videos/")
        self.BATCH_SIZE = 2
        self.EPOCHS = 50
        self.GEN_MODEL_PATH = join(self.cwdir,"model_gen_Conv2DLSTM_AAE")
        self.DIS_MODEL_PATH = join(self.cwdir,"model_dis_Conv2DLSTM_AAE")
        self.DEC_MODEL_PATH = join(self.cwdir,"model_dec_Conv2DLSTM_AAE")

        self.retrain = 0
        self.dim1 = 10
        self.dim2 = 256
        self.dim3 = 256
        self.dim4 = 1
        self.latent_dim = 327680
        self.curr_epoch = 0

    def reconfig(self, new_name, batch_size = 4, epochs = 5, retrain = 0, curr_epoch = 0):
        self.cwdir_name = new_name
        self.cwdir = join(self.data_dir, self.cwdir_name)
        self.run_data = join(self.cwdir, "training_dir")
        self.image_dir = join(self.run_data,self.data_set_name,"testing_videos/")

        self.BATCH_SIZE = batch_size
        self.EPOCHS = epochs
        self.GEN_MODEL_PATH = join(self.cwdir,"model_gen_Conv2DLSTM_AAE")
        self.DIS_MODEL_PATH = join(self.cwdir,"model_dis_Conv2DLSTM_AAE")
        self.DEC_MODEL_PATH = join(self.cwdir,"model_dec_Conv2DLSTM_AAE")

        self.retrain = retrain
        if retrain == 0:
            print("Configuring train from scratch")
            if not os.path.exists(self.cwdir):
                os.mkdir(self.cwdir)
                os.mkdir(self.run_data)
    
            if os.path.exists(self.run_data):
                shutil.rmtree(self.run_data)
                os.mkdir(self.run_data)
                os.makedirs(self.image_dir, exist_ok=True)
        else:
            if not os.path.exists(self.cwdir):
                os.mkdir(self.cwdir)
                os.mkdir(self.run_data)
                os.makedirs(self.image_dir, exist_ok=True)

        self.curr_epoch = curr_epoch

conf = Config(data_dir_="/content/drive/My Drive/", cwdir_name_="Conv2DLSTM_AAE_Avenue", data_set="AVDT/") 


In [0]:

def normalize_img(img):
    img = np.array(img, dtype=np.float32) / 256.0
    return img


def get_clips_by_stride(stride, frames_list, sequence_size):

    clips = []
    sz = len(frames_list)
    clip = np.zeros(shape=(sequence_size, 256, 256, 1))
    cnt = 0
    for start in range(0, stride):
        for i in range(start, sz, stride):
            clip[cnt, :, :, 0] = frames_list[i]
            cnt = cnt + 1
            if cnt == sequence_size:
                clips.append(clip)
                cnt = 0
    return clips

def get_clips_list(seq_size):

    clips = []
    # loop over the training folders (Train000,Train001,..)
    for f in sorted(listdir(conf.DATASET_PATH)):
        directory_path = join(conf.DATASET_PATH, f)
        if isdir(directory_path):
            all_frames = []
            # loop over all the images in the folder (0.tif,1.tif,..,199.tif)
            for c in sorted(listdir(directory_path)):
                img_path = join(directory_path, c)
                if str(img_path)[-3:] == "tif":
                    img = Image.open(img_path).resize((256, 256))

                    img = normalize_img(img)
                    all_frames.append(img)
            # get the 32-frames sequences from the list of images after applying data augmentation
            for stride in range(1, 3):
                clips.extend(get_clips_by_stride(stride=stride, frames_list=all_frames, sequence_size=seq_size))
    
    #print(np.array(clips).shape)
    return np.array(clips)


def cap_video(file_path):

    cap = cv2.VideoCapture(file_path)
    if (cap.isOpened()== False):
        print("Error opening video stream or file")

    frames = []
    while(cap.isOpened()):
        ret, frame = cap.read()
        if ret == True:
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            frame = cv2.resize(frame, (256, 256))
            frame = np.array(frame, dtype=np.float32) / 256.0
            frames.append(frame)
        else: 
            break
    cap.release()
    cv2.destroyAllWindows()
    return frames

def get_video_list():
    return  [join(conf.DATASET_PATH, x) for x in sorted(listdir(conf.DATASET_PATH))]
        

def get_clips_list_from_video(seq_size):

    clips = []
    for f in get_video_list():
        print("Loading video "+f)
        all_frames = []
        if(os.path.isfile(f)):
            all_frames = cap_video(f) 
            print(np.array(all_frames).shape)
        for stride in range(1, 2):
            strided = get_clips_by_stride(stride=stride, frames_list=all_frames, sequence_size=seq_size)
            #print(np.array(strided).shape)
            clips.extend(strided)

    return np.array(clips)

def get_test_sequences_from_video(test_case):
    test = np.array(cap_video(join(conf.TEST_DIR,test_case)))
    test = np.resize(test, (test.shape[0], conf.dim2, conf.dim3, conf.dim4))

    print("Test case loaded")
    sz = test.shape[0] - conf.dim1
    sequences = np.zeros((sz, conf.dim1, conf.dim2, conf.dim3, conf.dim4))
    # apply the sliding window technique to get the sequences
    for i in range(0, sz):
        clip = np.zeros((conf.dim1, conf.dim2, conf.dim3, conf.dim4))
        for j in range(0, conf.dim1):
            clip[j] = test[i + j, :, :, :]
        sequences[i] = clip
    return np.array(sequences)


def get_single_test(single_test_path):

    sz = 0
    for f in sorted(listdir(single_test_path)):
        if str(join(single_test_path, f))[-3:] == "tif":
          sz = sz +1

    test = np.zeros(shape=(sz, conf.dim2, conf.dim3, conf.dim4))
    cnt = 0
    for f in sorted(listdir(single_test_path)):
        if str(join(single_test_path, f))[-3:] == "tif":
            img = Image.open(join(single_test_path, f)).resize((conf.dim2, conf.dim3))
            #cv2_imshow(np.array(img,dtype=np.float32))
            #cv2.waitKey(0)
            img = normalize_img(img)
            test[cnt, :, :, 0] = img
            cnt = cnt + 1
    return test

def get_test_sequences(test_case_dir):
    test = get_single_test(join(conf.TEST_DIR,test_case_dir))
    print("Test case loaded")
    sz = test.shape[0] - conf.dim1
    sequences = np.zeros((sz, conf.dim1, conf.dim2, conf.dim3, conf.dim4))
    # apply the sliding window technique to get the sequences
    for i in range(0, sz):
        clip = np.zeros((conf.dim1, conf.dim2, conf.dim3, conf.dim4))
        for j in range(0, conf.dim1):
            clip[j] = test[i + j, :, :, :]
        sequences[i] = clip
    return sequences

def convert_images_back(image):
    return np.reshape(image,(256, 256))*256.0
    #print(image.shape)
    #return np.reshape(image[:, :, 0],(image.shape[0], image.shape[1]))*127.5 + 127.5




In [9]:
def build_model_enc(path=""):
    if conf.retrain == 1 and os.path.isfile(path):
        print("Loading enc from :"+path)
        model_gen=load_model(path, custom_objects={'LayerNormalization': LayerNormalization})
        return model_gen
    seq = Sequential()
    seq.add(TimeDistributed(Conv2D(128, (11, 11), strides=4, activation="relu", padding="same"), batch_input_shape=(None, 10, 256, 256, 1)))
    seq.add(LayerNormalization())
    seq.add(TimeDistributed(Conv2D(64, (5, 5), strides=2, activation="relu", padding="same")))
    seq.add(LayerNormalization())
    # # # # #
    seq.add(ConvLSTM2D(64, (3, 3), padding="same", return_sequences=True))
    seq.add(LayerNormalization())
    seq.add(ConvLSTM2D(32, (3, 3), padding="same", return_sequences=True))
    seq.add(LayerNormalization())
    seq.add(Flatten())
    seq.summary(line_length=150)
    return  seq

def build_model_dec(path=""):
    if conf.retrain == 1 and os.path.isfile(path):
        print("Loading dec from :"+path)
        model_gen=load_model(path, custom_objects={'LayerNormalization': LayerNormalization})
        return model_gen
    seq = Sequential()
    seq.add(Reshape((10, 32, 32, 32), input_shape=(conf.latent_dim,)))
    seq.add(ConvLSTM2D(32, (3, 3), padding="same", return_sequences=True))
    seq.add(LayerNormalization())
    seq.add(ConvLSTM2D(64, (3, 3), padding="same", return_sequences=True))
    seq.add(LayerNormalization())
    # # # # #
    seq.add(TimeDistributed(Conv2DTranspose(64, (5, 5), strides=2, activation="relu", padding="same")))
    seq.add(LayerNormalization())
    seq.add(TimeDistributed(Conv2DTranspose(128, (11, 11), strides=4, activation="relu", padding="same")))
    seq.add(LayerNormalization())
    seq.add(TimeDistributed(Conv2D(1, (11, 11), activation="sigmoid", padding="same")))
    seq.summary(line_length=150)
    return seq

def build_model_disc(path=""):
    if conf.retrain == 1 and os.path.isfile(path):
        print("Loading disc from :"+path)
        model_gen=load_model(path, custom_objects={'LayerNormalization': LayerNormalization})
        return model_gen
    seq = Sequential()
    model = Sequential()
    seq.add(Reshape((10, 32, 32, 32), input_shape=(conf.latent_dim,)))
    seq.add(TimeDistributed(Conv2D(16, (3, 3), strides=2, activation="relu", padding="same")))
    seq.add(MaxPool3D(pool_size=(2, 2, 2), strides=2, padding='valid'))
    seq.add(Flatten())
    seq.add(Dropout(0.4))
    seq.add(Dense(1, activation="sigmoid"))
    seq.summary(line_length=150)
    return seq

build_model_enc()
build_model_dec()
build_model_disc()

Model: "sequential_1"
______________________________________________________________________________________________________________________________________________________
Layer (type)                                                       Output Shape                                                Param #                
time_distributed_1 (TimeDistributed)                               (None, 10, 64, 64, 128)                                     15616                  
______________________________________________________________________________________________________________________________________________________
layer_normalization_1 (LayerNormalization)                         (None, 10, 64, 64, 128)                                     256                    
______________________________________________________________________________________________________________________________________________________
time_distributed_2 (TimeDistributed)                               (None

In [0]:
def build_model_aae(enc_path, dec_path, disc_path):
    model_enc = build_model_enc(enc_path)
    model_dec = build_model_dec(dec_path)
    model_disc = build_model_disc(disc_path)
    
    model_ae = Sequential()
    model_ae.add(model_enc)
    model_ae.add(model_dec)
    
    model_enc_disc = Sequential()
    model_enc_disc.add(model_enc)
    model_enc_disc.add(model_disc)
    
    model_enc.summary()
    model_dec.summary()
    model_disc.summary()
    model_ae.summary()
    model_enc_disc.summary()

    model_disc.compile(optimizer=Adam(lr=1e-4), loss="binary_crossentropy")
    model_enc_disc.compile(optimizer=Adam(lr=1e-4), loss="binary_crossentropy")
    model_ae.compile(optimizer=Adam(lr=1e-4, decay=1e-5, epsilon=1e-6), loss="mse")
    
    return model_enc, model_dec, model_disc, model_ae, model_enc_disc

 

In [0]:


def reconstruct_batch(model, sequences, epoch, folder_name):

    sz = sequences.shape[0]
    #InX = Input(shape=sequences.shape[1:]) 
    #tmpModel = Model(inputs=InX, outputs=model.get_layer("convTD5").output)
    reconstructed_sequences = model.predict(sequences,batch_size=conf.BATCH_SIZE)

    path = join(conf.run_data,folder_name, str(epoch)+"_epoch")
    os.makedirs(path, exist_ok=True)
    for i in range(0, sz):
        #cv2_imshow(np.reshape(reconstructed_sequences[i][2],(256, 256))*256)
        if i < 10:
            img_num = "00"+str(i)
        elif i < 100:
            img_num = "0"+str(i)
        else:
            img_num = str(i)
        print("Reconstructing : "+ str(reconstructed_sequences.shape))
        cv2.imwrite(join(path,"gen_"+img_num+".jpg"), convert_images_back(reconstructed_sequences[i][6]))
        
def settrainable(model, toset):
    for layer in model.layers:
        layer.trainable = toset
    model.trainable = toset




def evaluate_gen(sequences, model, test_case_dir, epoch=conf.EPOCHS):

    reconstructed_sequences = model.predict(sequences,batch_size=conf.BATCH_SIZE)
    
    sz = sequences.shape[0]
    
    print("Test size:"+str(sz))
   
    os.makedirs(join(conf.image_dir,test_case_dir,"epoch_"+str(epoch)), exist_ok=True)
    for i in range(0, sz):
        #print("sz "+str(i)+"\n")
        #cv2_imshow(np.reshape(reconstructed_sequences[i][2],(256, 256))*256)
        if i < 10:
            img_num = "00"+str(i)
        elif i < 100:
            img_num = "0"+str(i)
        else:
            img_num = str(i)    
        
        cv2.imwrite(join(conf.image_dir, test_case_dir, "epoch_"+str(epoch),"gen_"+img_num+".jpg"), convert_images_back(reconstructed_sequences[i][6]))


def evaluate_dis(sequences, model, test_case_dir, epoch=conf.EPOCHS):
    sr = model.predict(sequences,batch_size=conf.BATCH_SIZE)

    plt.plot(sr)
    plt.ylabel('regularity score Sr(t)')
    plt.xlabel('frame t')
    plt.savefig(join(conf.image_dir, test_case_dir, "epoch_"+str(epoch),"regularity_score.jpg"))
    plt.show()

def plot_history(d1_hist, d2_hist, d3_hist, image_name):
	# plot history
	plt.plot(d1_hist, label='ae_loss')
	plt.plot(d2_hist, label='disc_loss')
	plt.plot(d3_hist, label='enc_disc_loss')
	plt.legend()
	plt.savefig(image_name)
	plt.close()

In [12]:
import csv


test_case= "11.avi"
test_cases = get_test_sequences_from_video(test_case)
print("Test data set loaded: "+ str(test_cases.shape))
conf.reconfig(new_name="Conv2DLSTM_AAE_Avenue", batch_size=2, epochs=100, retrain=1, curr_epoch=0)
x_train = get_clips_list_from_video(conf.dim1)

print("Train data set loaded: "+str(x_train.shape))


model_enc, model_dec, model_disc, model_ae, model_enc_disc = build_model_aae(enc_path=conf.GEN_MODEL_PATH+str("ep"+str(conf.curr_epoch+1)), dec_path=conf.DEC_MODEL_PATH+str("ep"+str(conf.curr_epoch+1)), disc_path=conf.DIS_MODEL_PATH+str("ep"+str(conf.curr_epoch+1)))


batchsize = conf.BATCH_SIZE


for epochnumber in range(conf.curr_epoch, conf.EPOCHS):
    print("Epoch "+str(epochnumber+1))
    d1_hist, d2_hist, d3_hist = list(), list(), list()
    with open(join(conf.run_data, 'loss_val.csv'), mode='a') as csv_file:
        fieldnames = ['epoch_num', 'batch_num', 'ae_loss', 'disc_loss', 'enc_disc_loss']
        writer = csv.DictWriter(csv_file, fieldnames=fieldnames)
        writer.writeheader()
        for i in range(int(len(x_train) / batchsize)):
            print("================Starting batch "+str(i)+"===============")
            settrainable(model_ae, True)
            settrainable(model_enc, True)
            settrainable(model_dec, True)
        
            batch = x_train[i*batchsize:i*batchsize+batchsize]
            ae_loss = model_ae.train_on_batch(batch, batch)
            print("AE trained ")
            
            settrainable(model_disc, True)
            batchpred = model_enc.predict(batch)
            fakepred = np.random.standard_normal((batchsize, conf.latent_dim))
            #fakepred = np.random.normal(loc=0.0, scale=0.2, size=(batchsize, conf.latent_dim))
            discbatch_x = np.concatenate([batchpred, fakepred])
            discbatch_y = np.concatenate([np.zeros(batchsize), np.ones(batchsize)])
            disc_loss = model_disc.train_on_batch(discbatch_x, discbatch_y)
            print("Discriminator trained ")
            settrainable(model_enc_disc, True)
            settrainable(model_enc, True)
            settrainable(model_disc, False)
            enc_disc_loss = model_enc_disc.train_on_batch(batch, np.ones(batchsize))
            print("Encoder trained ")
            #print("ae_loss:"+str(ae_loss)+", disc_loss:"+str(disc_loss)+", enc_disc_loss:"+str(enc_disc_loss))
            d1_hist.append(ae_loss)
            d2_hist.append(disc_loss)
            d3_hist.append(enc_disc_loss)

            writer.writerow({'epoch_num': epochnumber, 'batch_num': i, 'ae_loss': ae_loss, 'disc_loss': disc_loss, 'enc_disc_loss': enc_disc_loss})
        csv_file.close()
    plot_history(d1_hist, d2_hist, d3_hist, join(conf.run_data,"plot_ep_"+str(epochnumber+1)))
    model_enc.save(conf.GEN_MODEL_PATH+str("ep"+str(epochnumber+1)))
    model_dec.save(conf.DEC_MODEL_PATH+str("ep"+str(epochnumber+1)))
    model_disc.save(conf.DIS_MODEL_PATH+str("ep"+str(epochnumber+1)))
    print("All models saved for epoch "+str(epochnumber+1))
    print("Reconstructing Gen ")
    evaluate_gen(test_cases, model_ae, test_case, epochnumber+1)
    #print("Reconstructing Dis")
    #evaluate_dis(test_cases, model_enc_disc, test_case, epochnumber+1)
    #print ("Reconstruction Loss:", model_ae.evaluate(x_train, x_train, verbose=0, batch_size=conf.BATCH_SIZE))
    #print ("Adverserial Loss:", model_enc_disc.evaluate(x_train, np.ones(len(x_train)), verbose=0, batch_size=conf.BATCH_SIZE))




Test case loaded
Test data set loaded: (462, 10, 256, 256, 1)
Loading video /content/drive/My Drive/AVDT/training_videos/01.avi
(1364, 256, 256)
Loading video /content/drive/My Drive/AVDT/training_videos/02.avi
(1511, 256, 256)
Loading video /content/drive/My Drive/AVDT/training_videos/03.avi
(1487, 256, 256)
Loading video /content/drive/My Drive/AVDT/training_videos/04.avi
(1511, 256, 256)
Loading video /content/drive/My Drive/AVDT/training_videos/05.avi
(815, 256, 256)
Loading video /content/drive/My Drive/AVDT/training_videos/06.avi
(1511, 256, 256)
Loading video /content/drive/My Drive/AVDT/training_videos/07.avi
(1099, 256, 256)
Loading video /content/drive/My Drive/AVDT/training_videos/08.avi
(1017, 256, 256)
Loading video /content/drive/My Drive/AVDT/training_videos/09.avi
(1391, 256, 256)
Loading video /content/drive/My Drive/AVDT/training_videos/10.avi
(1223, 256, 256)
Loading video /content/drive/My Drive/AVDT/training_videos/11.avi
(781, 256, 256)
Loading video /content/driv

/usr/local/lib/python3.6/dist-packages/keras/engine/training.py:297: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


Encoder trained 
================Starting batch 1===============
AE trained 
Discriminator trained 
Encoder trained 
================Starting batch 2===============
AE trained 
Discriminator trained 
Encoder trained 
================Starting batch 3===============
AE trained 
Discriminator trained 
Encoder trained 
================Starting batch 4===============
AE trained 
Discriminator trained 
Encoder trained 
================Starting batch 5===============
AE trained 
Discriminator trained 
Encoder trained 
================Starting batch 6===============
AE trained 
Discriminator trained 
Encoder trained 
================Starting batch 7===============
AE trained 
Discriminator trained 
Encoder trained 
================Starting batch 8===============
AE trained 
Discriminator trained 
Encoder trained 
================Starting batch 9===============
AE trained 
Discriminator trained 
Encoder trained 
================Starting batch 10===============
AE trained 
Discriminator trained 

KeyboardInterrupt: ignored